## Imports 

In [1]:
import pandas as pd
import inflection
import math
import numpy as np
import seaborn as sns
from IPython.core.display import HTML
import matplotlib.pyplot as plt
from IPython.display import Image
import datetime

### Loading data

In [2]:
df_sales_raw= pd.read_csv('train.csv',low_memory= False)
df_store_raw= pd.read_csv('store.csv',low_memory= False)
# Fazendo o merge dos dados para transformar os dois datasets em apenas 1 
df_raw=pd.merge(df_sales_raw,df_store_raw,how='left', on='Store')


In [3]:
df_raw.sample()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
265601,291,7,2014-11-30,0,0,0,0,0,0,d,a,NaN,NaN,NaN,0,NaN,NaN,NaN


## 1. Descrição dos dados


### 1.1. Renomeando as colunas 

In [4]:
df1=df_raw.copy()  #criando uma copia do data para seguranca dos dados originais


In [5]:
df1.columns      #olhando os nomes das colunas

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval'],
      dtype='object')

In [6]:
cols_old=['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval']
snakecase=lambda x:inflection.underscore(x) 
cols_new=list(map(snakecase,cols_old))  #lambda: e a declaracao de uma funcao 
#snakecase: reencreve em minusculo e com _ como separador
#funcao map: faz o mapeamento da funcao snakecase em todas as palavras da lista cols_old

#Rename
df1.columns=cols_new


### 1.2 Dimensao dos dados

In [7]:
print('Numbers of Rows:{}'.format(df1.shape[0])) 
print('Numbers of Columns:{}'.format(df1.shape[1]))

Numbers of Rows:1017209
Numbers of Columns:18


### 1.3. Tipo de dados 

In [8]:
df1.dtypes

store                             int64
day_of_week                       int64
date                             object
sales                             int64
customers                         int64
open                              int64
promo                             int64
state_holiday                    object
school_holiday                    int64
store_type                       object
assortment                       object
competition_distance            float64
competition_open_since_month    float64
competition_open_since_year     float64
promo2                            int64
promo2_since_week               float64
promo2_since_year               float64
promo_interval                   object
dtype: object

In [9]:
df1['date']=pd.to_datetime(df1['date'])  # ajustando a variavel de data 



### 1.4 Checando os NAs

In [10]:
df1.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

### 1.4 Preenchendo os NAs

In [11]:
# opcao 1. excluir os NAs /// opcao 2. preencher com a media /// opcao 3. pensar como quem entende do negocio 

#competition_distance (Na's faltantes:  2642 )

df1['competition_distance']=df1['competition_distance'].apply(lambda x:2000000.0 if math.isnan(x) else x)
# distancia em metros do competidor mais proximo . 
# math.isnam : encontra na coluna os valores sem dados 



In [12]:
df1['competition_distance'].max()

2000000.0

In [13]:
df1.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance                 0
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

In [14]:
df1.sample()

,store,day_of_week,date,sales,customers,open,promo,state_holiday,school_holiday,store_type,assortment,competition_distance,competition_open_since_month,competition_open_since_year,promo2,promo2_since_week,promo2_since_year,promo_interval
380557,232,3,2014-07-30,3823,330,1,1,0,1,c,c,13570.0,5.0,2010.0,1,10.0,2013.0,"Mar,Jun,Sept,Dec"


In [15]:

#nao possui competidor proximo ou nao tem o dado
#No entendimento assumo que : se tiver dados vazios(isnan.math) na coluna 'competition since mounth/year/week', eu vou pegar a coluna 'date' extrair o mes e substituindo o NA, aplicando(apply)em todas as colunas (axis 1)
    
#competition_open_since_month (Na's faltantes: 323348)
#mes e ano que o competidor abriu 
df1['competition_open_since_month']=df1.apply(lambda x:x ['date'].month if math.isnan (x['competition_open_since_month'])else x ['competition_open_since_month'],axis=1)


#competition_open_since_year   ( Na's faltantes:  323348)
df1['competition_open_since_year']=df1.apply(lambda x:x ['date'].year if math.isnan (x['competition_open_since_year'])else x ['competition_open_since_year'],axis=1)

#promo2_since_week             ( Na's faltantes:  508031)
df1['promo2_since_week']=df1.apply(lambda x:x ['date'].week if math.isnan (x['promo2_since_week'])else x ['promo2_since_week'],axis=1)

#promo2_since_year              Na's faltantes:  508031
df1['promo2_since_year']=df1.apply(lambda x:x ['date'].year if math.isnan (x['promo2_since_year'])else x ['promo2_since_year'],axis=1)

In [18]:
#promo_interval                  508031
#crio uma lista de meses , se a data estiver dentro da lista a promo2 foi ativa.

month_map={1:'Jan',2:'Fev',3:'Mar',4:'Apr',5:'May',6:'Jun',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'}
df1['promo_interval'].fillna(0,inplace=True)

df1['month_map']=df1['date'].dt.month.map(month_map)
df1['is_promo']=df1[['promo_interval','month_map']].apply(lambda x:0 if x['promo_interval']==0 else 1if x['month_map']in x['promo_interval'].split(',')else 0, axis=1)                                      
